## Conectando à DataBase

### Importando bibliotecas

In [155]:
import psycopg2
import joblib
import json
import numpy as np
from io import BytesIO

### Configurações da DataBase

In [156]:
DB_CONFIG = {
    'dbname': 'ml_models_comparations',
    'user': 'postgres',
    'password': 'zzrispudb',
    'host': 'localhost',
    'port': '5432'
}

### Função para salvar o modelo na DataBase

In [157]:
def save_model_to_db(model, name, model_type, hyperparameters):
    model_bytes = BytesIO()
    joblib.dump(model, model_bytes)
    model_bytes.seek(0)

    conn = psycopg2.connect(**DB_CONFIG, options='-c client_encoding=utf8')
    cur = conn.cursor()

    try:
        hyperparameters = json.dumps(hyperparameters, ensure_ascii=False)
    except TypeError as e:
        print(f'Error: {e}')
        hyperparameters = "{}"

    cur.execute(
        "INSERT INTO models (name, type, hyperparameters) VALUES (%s, %s, %s) RETURNING id",
        (name, model_type, json.dumps(hyperparameters))
    )

    model_id = cur.fetchone()[0]
    conn.commit()
    cur.close()
    conn.close()
    
    return model_id

### Função para salvar o preprocessamento na DataBase

In [158]:
def save_preprocessing_to_db(sampling_strategy, normalized, split_method, test_size):
    conn = psycopg2.connect(**DB_CONFIG, options='-c client_encoding=utf8')
    cur = conn.cursor()

    cur.execute(
        "INSERT INTO preprocessing (sampling_strategy, normalized, split_method, test_size) VALUES (%s, %s, %s, %s) RETURNING id;",
        (sampling_strategy, normalized, split_method, test_size)
    )

    preprocessing_id = cur.fetchone()[0]
    conn.commit()
    cur.close()
    conn.close()
    
    return preprocessing_id

### Função para salvar as métricas na DataBase

In [159]:
def save_metrics_to_db(result_id, metrics):
    conn = psycopg2.connect(**DB_CONFIG, options='-c client_encoding=utf8')
    cur = conn.cursor()

    for metric_name, metric_value in metrics.items():
        cur.execute(
            "INSERT INTO metrics (result_id, metric_name, metric_value) VALUES (%s, %s, %s);",
            (result_id, metric_name, float(metric_value))
        )

    conn.commit()
    cur.close()
    conn.close()

### Função geral para salvar os resultados na DataBase

In [160]:
def save_results(model, name, model_type, hyperparameters, sampling_strategy, normalized, split_method, test_size, dataset, train_size, metrics):
    model_id = save_model_to_db(model, name, model_type, hyperparameters)
    preprocessing_id = save_preprocessing_to_db(sampling_strategy, normalized, split_method, test_size)

    conn = psycopg2.connect(**DB_CONFIG, options='-c client_encoding=utf8')
    cur = conn.cursor()

    cur.execute(
        "INSERT INTO results (model_id, preprocessing_id, dataset, train_size, test_size) VALUES (%s, %s, %s, %s, %s) RETURNING id;",
        (model_id, preprocessing_id, dataset, train_size, test_size)
    )

    result_id = cur.fetchone()[0]
    conn.commit()
    cur.close()
    conn.close()

    save_metrics_to_db(result_id, metrics)

## Estrutura do Pipeline
### Importando bibliotecas

In [161]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedGroupKFold, KFold
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

### Pré-processamento de dados

Amostragem e separação em treino/validação/teste.

> Amostragem:
> * Urdersampling (reduz classes majoritárias)
> * Oversampling (reduz classes minoritárias)
> * Stratified Sampling (mantém proporções das classes)

> Divisão de dados:
> * Hold-out: (treino/validação/teste fix, ex: 70/20/10)
> * Cross-Validation (validação cruzada com K-Folds)

In [162]:
def preprocess_data(X, y, problem_type='classification', sampling_strategy='none', split_method='holdout', test_size=0.2, folds=5, random_state=42):
    # Converter para DataFrame se for numpy array
    if not isinstance(X, pd.DataFrame):
        X = pd.DataFrame(X)

    # Aplicar amostragem se necessário (só para classificação)
    if sampling_strategy == 'undersampling':
        X, y = RandomUnderSampler().fit_resample(X, y)
    elif sampling_strategy == 'oversampling':
        X, y = SMOTE().fit_resample(X, y)

    # Dividir os dados em treino e teste
    if split_method == 'holdout':
        stratify = y if problem_type == 'classification' else None
        return train_test_split(X, y, test_size=test_size, stratify=stratify, random_state=random_state)
    elif split_method == 'cross-validation':
        if problem_type == 'classification':
            return StratifiedGroupKFold(n_splits=folds, shuffle=True, random_state=random_state).split(X, y, groups=X['subject'])
        else:
            return KFold(n_splits=folds, shuffle=True, random_state=random_state).split(X, y)
    
    return X, y

### Carregando DataSets

> Classificação:
> * Iris DataSet
> * Breast Cancer Dataset

> Regressão
> * Boston Housing Dataset
> * California Housing Dataset

#### Iris DataSet

In [163]:
# Iris DataSet
from sklearn.datasets import load_iris

iris_ds = load_iris()
X_iris, y_iris = iris_ds.data, iris_ds.target

X_train_iris, X_test_iris, y_train_iris, y_test_iris = preprocess_data(X_iris, y_iris, sampling_strategy='oversampling', split_method='holdout')

print(f"Tamanho do treino: {len(X_train_iris)} | Tamanho do teste: {len(X_test_iris)}")

Tamanho do treino: 120 | Tamanho do teste: 30


#### Breast Cancer DataSet

In [164]:
# Breast Cancer DataSet
from sklearn.datasets import load_breast_cancer

bc_ds = load_breast_cancer()
X_bc, y_bc = bc_ds.data, bc_ds.target

X_train_bc, X_test_bc, y_train_bc, y_test_bc = preprocess_data(X_bc, y_bc, sampling_strategy='undersampling', split_method='holdout')
print(f"Tamanho do treino: {len(X_train_bc)} | Tamanho do teste: {len(X_test_bc)}")

Tamanho do treino: 339 | Tamanho do teste: 85


#### Boston Housing DataSet

In [165]:
# Boston Housing DataSet
bh_ds = pd.read_csv('https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv')

X_bh, y_bh = bh_ds.drop(columns=["medv"]), bh_ds['medv']

X_train_bh, X_test_bh, y_train_bh, y_test_bh = preprocess_data(X_bh, y_bh, problem_type='regression', sampling_strategy='none', split_method='holdout')
print(f"Tamanho do treino: {len(X_train_bh)} | Tamanho do teste: {len(X_test_bh)}")
print(y_test_bc)

Tamanho do treino: 404 | Tamanho do teste: 102
[1 1 0 0 1 0 1 1 0 1 0 1 1 0 1 0 1 1 1 1 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 0 0
 0 1 0 0 1 1 0 1 0 0 0 1 0 1 0 1 1 1 0 0 1 0 1 1 1 0 1 0 1 1 0 0 1 1 0 1 0
 0 0 0 1 0 1 0 1 1 1 1]


#### California Housing DataSet

In [166]:
# California Housing DataSet
from sklearn.datasets import fetch_california_housing

cali_ds = fetch_california_housing(as_frame=True)

X_cali, y_cali = cali_ds.data, cali_ds.target

X_train_cali, X_test_cali, y_train_cali, y_test_cali = preprocess_data(X_cali, y_cali, problem_type='regression', sampling_strategy='none', split_method='holdout')
print(f"Tamanho do treino: {len(X_train_cali)} | Tamanho do teste: {len(X_test_cali)}")

Tamanho do treino: 16512 | Tamanho do teste: 4128


## Treinando e Avaliando métricas dos modelos

### Importando modelos de ML

> Classificação:
> * Random Forest Classifier
> * Decision Tree Classifier

> Regressão:
> * Linear Regression
> * Decision Tree Regressor

In [167]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

### Importando métodos de métricas para ML

> Classificação:
> * Acurácia
> * Precisão
> * Recall
> * F1-score
> * ROC-AUC

> Regressão:
> * MSE
> * RMSE
> * MAE
> * R²

In [168]:
# Para classificação
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
# Para regressão
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error, r2_score

### Construindo funções para treinar e avaliar modelos, e salvar resultados na DataBase
#### Modelos de classificação

In [169]:
# Função para treinar e avaliar modelos de classificação
def train_evaluate_saveToDB_classification(dataset_name, X_train, y_train, X_test, y_test, normalized=False):
    models = {
        'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
        'DecisionTree': DecisionTreeClassifier(max_depth=5, random_state=42)
    }

    results = {}
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        if len(np.unique(y_test)) == 2:
            roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
        else:
            roc_auc = roc_auc_score(y_test, model.predict_proba(X_test), multi_class='ovo', average='weighted')

        results[name] = {
            'accuracy': accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred, average='weighted'),
            'recall': recall_score(y_test, y_pred, average='weighted'),
            'f1': f1_score(y_test, y_pred, average='weighted'),
            'roc_auc': roc_auc
        }

        save_results(model, name, 'classification', model.get_params(), 'oversampling', normalized, 'holdout', len(X_test), dataset_name, len(X_train), results[name])

    return results

#### Modelos de regressão

In [170]:
# Função para treinar e avaliar modelos de regressão
def train_evaluate_saveToDB_regression(dataset_name, X_train, y_train, X_test, y_test, normalized=False):
    models = {
        'LinearRegression': LinearRegression(),
        'DecisionTree': DecisionTreeRegressor()
    }

    results = {}
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        results[name] = {
            'mse': mean_squared_error(y_test, y_pred),
            'mape': mean_absolute_percentage_error(y_test, y_pred),
            'mae': mean_absolute_error(y_test, y_pred),
            'r2': r2_score(y_test, y_pred)
        }
        
        save_results(model, name, 'regression', model.get_params(), 'none', normalized, 'holdout', len(X_test), dataset_name, len(X_train), results[name])

    return results

### Treinando e avaliando diferentes modelos com diferentes DataSets, e salvando seus resultados na DataBase

#### Iris Data Set

In [171]:
iris_results_raw = train_evaluate_saveToDB_classification("Iris DataSet", X_train_iris, y_train_iris, X_test_iris, y_test_iris)
iris_results_normalized = train_evaluate_saveToDB_classification("Iris DataSet", StandardScaler().fit_transform(X_train_iris), y_train_iris, StandardScaler().fit_transform(X_test_iris), y_test_iris, True)
print("Iris DataSet | Não normalizado\n", pd.DataFrame.from_dict(iris_results_raw))
print("\nIris DataSet | Normalizado\n", pd.DataFrame.from_dict(iris_results_normalized))

Iris DataSet | Não normalizado
            RandomForest  DecisionTree
accuracy       0.900000      0.933333
precision      0.902357      0.933333
recall         0.900000      0.933333
f1             0.899749      0.933333
roc_auc        0.986667      0.950000

Iris DataSet | Normalizado
            RandomForest  DecisionTree
accuracy       0.966667      0.933333
precision      0.969697      0.933333
recall         0.966667      0.933333
f1             0.966583      0.933333
roc_auc        0.986667      0.950000


#### Breast Cancer DataSet

In [172]:
bc_results_raw = train_evaluate_saveToDB_classification("Breast Cancer DataSet", X_train_bc, y_train_bc, X_test_bc, y_test_bc)
bc_results_normalized = train_evaluate_saveToDB_classification("Breast Cancer DataSet", StandardScaler().fit_transform(X_train_bc), y_train_bc, StandardScaler().fit_transform(X_test_bc), y_test_bc, True)
print("Breast Cancer DataSet | Não normalizado\n", pd.DataFrame.from_dict(bc_results_raw))
print("\nBreast Cancer DataSet | Normalizado\n", pd.DataFrame.from_dict(bc_results_normalized))

Breast Cancer DataSet | Não normalizado
            RandomForest  DecisionTree
accuracy       0.917647      0.964706
precision      0.917901      0.964973
recall         0.917647      0.964706
f1             0.917647      0.964706
roc_auc        0.992248      0.955703

Breast Cancer DataSet | Normalizado
            RandomForest  DecisionTree
accuracy       0.917647      0.952941
precision      0.917901      0.953972
recall         0.917647      0.952941
f1             0.917647      0.952928
roc_auc        0.991141      0.945183


#### Bonston Housing DataSet

In [173]:
bh_results_raw = train_evaluate_saveToDB_regression("Bonston Housing DataSet", X_train_bh, y_train_bh, X_test_bh, y_test_bh)
bh_results_normalized = train_evaluate_saveToDB_regression("Bonston Housing DataSet", StandardScaler().fit_transform(X_train_bh), y_train_bh, StandardScaler().fit_transform(X_test_bh), y_test_bh, True)
print("Boston Housing DataSet | Não normalizado\n", pd.DataFrame.from_dict(bh_results_raw))
print("\nBoston Housing DataSet | Normalizado\n", pd.DataFrame.from_dict(bh_results_normalized))

Boston Housing DataSet | Não normalizado
       LinearRegression  DecisionTree
mse          24.291119     21.508922
mape          0.168664      0.146610
mae           3.189092      2.673529
r2            0.668759      0.706698

Boston Housing DataSet | Normalizado
       LinearRegression  DecisionTree
mse          27.406836     30.907843
mape          0.207548      0.191986
mae           3.852622      3.554902
r2            0.626273      0.578532


#### California Housing DataSet

In [174]:
cali_results_raw = train_evaluate_saveToDB_regression("California Housing DataSet", X_train_cali, y_train_cali, X_test_cali, y_test_cali)
cali_results_normalized = train_evaluate_saveToDB_regression("California Housing DataSet", StandardScaler().fit_transform(X_train_cali), y_train_cali, StandardScaler().fit_transform(X_test_cali), y_test_cali, True)
print("California Housing DataSet | Não normalizado\n", pd.DataFrame.from_dict(cali_results_raw))
print("\nCalifornia Housing DataSet | Normalizado\n", pd.DataFrame.from_dict(cali_results_normalized))

California Housing DataSet | Não normalizado
       LinearRegression  DecisionTree
mse           0.555892      0.484998
mape          0.319522      0.244687
mae           0.533200      0.450528
r2            0.575788      0.629888

California Housing DataSet | Normalizado
       LinearRegression  DecisionTree
mse           0.538865      1.219295
mape          0.321809      0.419667
mae           0.535317      0.744401
r2            0.588781      0.069531
